# 4. Numerically Check Sufficiency of Self-Reactive 

In [1]:
import numpy as np

import matplotlib.pyplot as plt

import tqdm

In [6]:
import repeated_play

In [2]:
import sympy as sym

sym.init_printing(True)

**Against a Reactive Player, Self-Reactive Works**

If Y is playing a strategy with a memory of N rounds, while X is playing a strategy with a memory of only 1 round, then by averaging over the resulting probability distribution of all sequences of N outcomes, we can produce an alternative strategy for Y with a memory of only 1 round. This alternative strategy's long-run average scores against X's strategy are identical.

## Numerically Check the Statement

### Reactive-1

In [15]:
for i in range(10**3):
    np.random.seed(i)

    # a random reactive strategy
    q_val = np.random.random(2)

    # a random memory one strategy
    p_val = np.random.random(4)

    # invariant distribution v

    M = repeated_play.transition_matrix_repeated_game(
        np.concatenate((q_val, q_val)), p_val, memory="one", analytical=False
    )

    ss = repeated_play.stationary_distribution(M)[0]

    # equations for self reactive

    p1 = (ss[0] * p_val[0] + ss[2] * p_val[1]) / (ss[0] + ss[2])

    p2 = (ss[1] * p_val[2] + ss[3] * p_val[3]) / (ss[1] + ss[3])

    # new invariant

    M2 = repeated_play.transition_matrix_repeated_game(
        np.concatenate((q_val, q_val)),
        [p1, p1, p2, p2],
        memory="one",
        analytical=False,
    )

    ss2 = repeated_play.stationary_distribution(M2)[0]

    # compare them

    assert np.isclose(ss2, ss).all()

### Reactive-2

### The Press and Dyson way: https://www.pnas.org/doi/abs/10.1073/pnas.1206569109

In [24]:
cc = [0, 2, 8, 10]

cd = [1, 3, 9, 11]

dc = [4, 6, 12, 14]

dd = [5, 7, 13, 15]

cc_player = [0, 1, 4, 5]

cd_player = [2, 3, 6, 7]

dc_player = [8, 9, 12, 13]

dd_player = [10, 11, 14, 15]

In [25]:
# b, c = 3, 1

r, s, t, p = 3, 0, 5, 1

for i in range(2, 10**3):
    np.random.seed(i)

    # a random reactive strategy
    q_1val, q_2val, q_3val, q_4val = np.random.random(4)

    q_player = [
        q_1val,
        q_2val,
        q_1val,
        q_2val,
        q_3val,
        q_4val,
        q_3val,
        q_4val,
        q_1val,
        q_2val,
        q_1val,
        q_2val,
        q_3val,
        q_4val,
        q_3val,
        q_4val,
    ]

    # a random memory one strategy
    player = np.random.random(16)

    # invariant distribution v

    M = repeated_play.transition_matrix_repeated_game(
        q_player, player, memory="two", analytical=False
    )

    ss = repeated_play.stationary_distribution(M)[0]

    # equations for self reactive

    p1 = sum([ss[j] * player[l] for j, l in zip(cc, cc_player)]) / sum(
        [ss[j] for j in cc]
    )

    p2 = sum([ss[j] * player[l] for j, l in zip(cd, cd_player)]) / sum(
        [ss[j] for j in cd]
    )

    p3 = sum([ss[j] * player[l] for j, l in zip(dc, dc_player)]) / sum(
        [ss[j] for j in dc]
    )

    p4 = sum([ss[j] * player[l] for j, l in zip(dd, dd_player)]) / sum(
        [ss[j] for j in dd]
    )

    # new invariant

    M2 = repeated_play.transition_matrix_repeated_game(
        q_player,
        [p1, p1, p2, p2, p1, p1, p2, p2, p3, p3, p4, p4, p3, p3, p4, p4],
        memory="two",
        analytical=False,
    )

    ss2 = repeated_play.stationary_distribution(M2)[0]

    # compare them

    assert np.isclose(
        ss2 @ np.array([r, s, t, p] * 4), ss @ np.array([r, s, t, p] * 4)
    )

### The Peter Park way: https://www.nature.com/articles/s41467-022-28336-2

In [28]:
cc = [0, 1, 4, 5]

cd = [2, 3, 6, 7]

dc = [8, 9, 12, 13]

dd = [10, 11, 14, 15]

In [29]:
b, c = 3, 1

for i in range(1, 10**3):
    np.random.seed(i)

    # a random reactive strategy
    q_1val, q_2val, q_3val, q_4val = np.random.random(4)

    q_player = [
        q_1val,
        q_2val,
        q_1val,
        q_2val,
        q_3val,
        q_4val,
        q_3val,
        q_4val,
        q_1val,
        q_2val,
        q_1val,
        q_2val,
        q_3val,
        q_4val,
        q_3val,
        q_4val,
    ]

    # a random memory one strategy
    p_val = np.random.random(16)

    # invariant distribution v

    M = repeated_play.transition_matrix_repeated_game(
        p_val, q_player, memory="two", analytical=False
    )

    ss = repeated_play.stationary_distribution(M)[0]

    # equations for self reactive

    p1 = sum([ss[j] * p_val[j] for j in cc]) / sum([ss[j] for j in cc])

    p2 = sum([ss[j] * p_val[j] for j in cd]) / sum([ss[j] for j in cd])

    p3 = sum([ss[j] * p_val[j] for j in dc]) / sum([ss[j] for j in dc])

    p4 = sum([ss[j] * p_val[j] for j in dd]) / sum([ss[j] for j in dd])

    # new invariant

    M2 = repeated_play.transition_matrix_repeated_game(
        [p1, p1, p2, p2, p1, p1, p2, p2, p3, p3, p4, p4, p3, p3, p4, p4],
        q_player,
        memory="two",
        analytical=False,
    )

    ss2 = repeated_play.stationary_distribution(M2)[0]

    # compare them

    assert np.isclose(
        ss @ np.array([b - c, -c, b, 0] * 4),
        ss2 @ np.array([b - c, -c, b, 0] * 4),
    ).all()

#### Further checks with conditions


1. $u^{p}(cc)$ = $u^{p'}(cc)$ 

2. $u^{p}(cd)$ = $u^{p'}(cd)$ 

3. $u^{p}(dc)$ = $u^{p'}(dc)$

4. $u^{p}(dd)$ = $u^{p'}(dd)$ 

In [30]:
b, c = 3, 1

for i in range(1, 10**3):
    np.random.seed(i)

    # a random reactive strategy
    q_1val, q_2val, q_3val, q_4val = np.random.random(4)

    q_player = [
        q_1val,
        q_2val,
        q_1val,
        q_2val,
        q_3val,
        q_4val,
        q_3val,
        q_4val,
        q_1val,
        q_2val,
        q_1val,
        q_2val,
        q_3val,
        q_4val,
        q_3val,
        q_4val,
    ]

    # a random memory one strategy
    p_val = np.random.random(16)

    # invariant distribution v

    M = repeated_play.transition_matrix_repeated_game(
        p_val, q_player, memory="two", analytical=False
    )

    ss = repeated_play.stationary_distribution(M)[0]

    # equations for self reactive

    p1 = sum([ss[j] * p_val[j] for j in cc]) / sum([ss[j] for j in cc])

    p2 = sum([ss[j] * p_val[j] for j in cd]) / sum([ss[j] for j in cd])

    p3 = sum([ss[j] * p_val[j] for j in dc]) / sum([ss[j] for j in dc])

    p4 = sum([ss[j] * p_val[j] for j in dd]) / sum([ss[j] for j in dd])

    # new invariant

    M2 = repeated_play.transition_matrix_repeated_game(
        [p1, p1, p2, p2, p1, p1, p2, p2, p3, p3, p4, p4, p3, p3, p4, p4],
        q_player,
        memory="two",
        analytical=False,
    )

    ss2 = repeated_play.stationary_distribution(M2)[0]

    assert np.isclose(sum([ss2[i] for i in cc]), sum([ss[i] for i in cc]))

    assert np.isclose(sum([ss2[i] for i in cd]), sum([ss[i] for i in cd]))

    assert np.isclose(sum([ss2[i] for i in dc]), sum([ss[i] for i in dc]))

    assert np.isclose(sum([ss2[i] for i in dd]), sum([ss[i] for i in dd]))